In [ ]:
import http.client
import json
import time
from kafka import KafkaProducer

# Configuration du Kafka Producer
kafka_broker = "kafka:9092"
kafka_topic = "leboncoin2"

producer = KafkaProducer(bootstrap_servers=kafka_broker,
                         value_serializer=lambda m: json.dumps(m).encode('ascii'))

# Configuration de l'API Le Bon Coin
conn = http.client.HTTPSConnection("leboncoin1.p.rapidapi.com")
headers = {
    'X-RapidAPI-Key': "75d8e9fcbdmshe1c9e3402676a93p103d36jsn22432e5b1526",
    'X-RapidAPI-Host': "leboncoin1.p.rapidapi.com"
}

# Parcourir les pages de l'API Le Bon Coin
for page in range(1, 30):
    query = f"https%3A%2F%2Fwww.leboncoin.fr%2Flocations%2Foffres%2Fp-{page}"
    conn.request("GET", f"/v2/leboncoin/search?query={query}", headers=headers)
    res = conn.getresponse()
    data = res.read()

    if res.status != 200:
        print(f"Error {res.status}: {res.reason}")
        break

    # Charger les données JSON et envoyer chaque annonce individuellement au topic Kafka
    json_data = json.loads(data.decode("utf-8"))
    ads_list = json_data['ads']
    for ad in ads_list:
        producer.send(kafka_topic, ad)
        print(f"Annonce envoyée au topic Kafka : {ad}")

    # Ajouter un délai de 5 secondes avant d'envoyer la prochaine requête
    time.sleep(5)
